# Forms

> FastHTML form components for API key input and management

In [ ]:
#| default_exp components.forms

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Dict, Any, Callable
from fasthtml.common import Form, Div, Input, Label, Button, Span, P, Select, Option, Details, Summary

# daisyUI imports
from cjm_fasthtml_daisyui.components.data_input.text_input import text_input, text_input_colors, text_input_sizes
from cjm_fasthtml_daisyui.components.data_input.fieldset import fieldset, label as fieldset_label
from cjm_fasthtml_daisyui.components.data_input.select import select, select_colors, select_sizes
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_sizes, btn_modifiers
from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title, card_actions
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui, ring_dui

# Tailwind imports
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, flex_display, flex_direction, grow, items, justify
from cjm_fasthtml_tailwind.utilities.spacing import space, m, p
from cjm_fasthtml_tailwind.utilities.sizing import w, max_w, min_w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, leading, font_family
from cjm_fasthtml_tailwind.utilities.effects import shadow, ring
from cjm_fasthtml_tailwind.utilities.borders import rounded, border
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_byok.utils.helpers import get_provider_info, format_provider_name
from cjm_fasthtml_byok.core.security import mask_key

In [ ]:
#| export
from fasthtml.common import Div
from fasthtml.jupyter import JupyUvi, HTMX
from cjm_fasthtml_daisyui.core.testing import create_test_app, create_test_page, start_test_server
from cjm_fasthtml_daisyui.core.themes import DaisyUITheme
from IPython.display import display

## Basic Key Input Form

In [ ]:
#| export
def KeyInputForm(
    provider: str,  # The API provider identifier
    action: Optional[str] = None,  # Form action URL (defaults to /api/keys/{provider})
    method: str = "post",  # HTTP method (default: "post")
    show_help: bool = True,  # Whether to show help text
    custom_placeholder: Optional[str] = None,  # Custom placeholder text
    extra_fields: Optional[List[tuple]] = None,  # Additional form fields as [(name, type, placeholder, required), ...]
    provider_config: Optional[Dict[str, Any]] = None  # Optional provider configuration
) -> Form:  # FastHTML Form component
    "Create a form for inputting an API key with improved design."
    provider_info = get_provider_info(provider, provider_config)
    placeholder = custom_placeholder or provider_info['placeholder']
    action_url = action or f"/api/keys/{provider}"
    
    form_fields = []
    
    # Main API key field with better visual hierarchy
    form_fields.append(
        Div(
            Label(
                Span(
                    f"{provider_info['name']} API Key",
                    cls=combine_classes(
                        font_weight.medium,
                        text_dui.base_content  # Semantic color for label
                    )
                ),
                Input(
                    type="password",
                    name="api_key",
                    placeholder=placeholder,
                    cls=combine_classes(
                        text_input,
                        text_input_sizes.md,  # Consistent sizing
                        w.full,
                        bg_dui.base_100,  # Theme-aware background
                        transition.all,  # Smooth transitions
                        duration._200,
                        text_input.focus,  # Focus state
                        ring(2).focus,  # Focus ring
                        ring_dui.primary.opacity(50).focus  # Primary color focus ring
                    ),
                    required=True,
                    autocomplete="off"
                ),
                cls=combine_classes(flex_display, flex_direction.col, gap(2))
            ),
            P(
                "Your API key will be encrypted and stored securely" if show_help else "",
                cls=combine_classes(
                    font_size.sm,
                    text_dui.base_content.opacity(60),  # Semantic color with opacity
                    m.t(1)
                ) if show_help else None
            ) if show_help else None,
            cls=combine_classes(space.y(2))  # Better spacing
        )
    )
    
    # Add extra fields if provided with consistent styling
    if extra_fields:
        for field_name, field_type, field_placeholder, field_required in extra_fields:
            form_fields.append(
                Div(
                    Label(
                        Span(
                            field_name.replace('_', ' ').title(),
                            cls=combine_classes(
                                font_weight.medium,
                                text_dui.base_content
                            )
                        ),
                        Input(
                            type=field_type,
                            name=field_name,
                            placeholder=field_placeholder,
                            cls=combine_classes(
                                text_input,
                                text_input_sizes.md,
                                w.full,
                                bg_dui.base_100,
                                transition.all,
                                duration._200
                            ),
                            required=field_required
                        ),
                        cls=combine_classes(flex_display, flex_direction.col, gap(2))
                    ),
                    cls=combine_classes(space.y(2))
                )
            )
    
    # Enhanced submit button with better visual weight
    form_fields.append(
        Button(
            "Save Key",
            type="submit",
            cls=combine_classes(
                btn,
                btn_colors.primary,
                btn_sizes.md,  # Consistent size
                w.full,  # Full width for better mobile experience
                shadow.md,  # Add depth
                transition.all,
                duration._200,
                shadow.lg.hover,  # Elevate on hover
                m.t(4)  # More spacing before button
            )
        )
    )
    
    return Form(
        *form_fields,
        method=method,
        action=action_url,
        cls=combine_classes(
            flex_display,
            flex_direction.col,
            gap(6),  # More generous spacing
            max_w.md,  # Constrain width for better readability
            w.full
        )
    )

In [ ]:
#| eval: false
#| output: false
test_func = KeyInputForm
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Test with a custom provider
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), 
                           test_func(provider="my_custom_service"))
server = start_test_server(app)
display(HTMX())
server.stop()

## Multi-Provider Key Form

In [ ]:
#| export
def MultiProviderKeyForm(
    providers: List[str],  # List of provider identifiers
    action: str = "/api/keys",  # Form action URL
    method: str = "post",  # HTTP method
    default_provider: Optional[str] = None,  # Initially selected provider
    provider_config: Optional[Dict[str, Any]] = None  # Optional provider configuration
) -> Form:  # FastHTML Form component with provider selection
    "Create a form that allows selecting from multiple providers with enhanced UX."
    default = default_provider or (providers[0] if providers else None)
    
    return Form(
        # Card-like container for better visual grouping
        Div(
            # Provider selection with improved styling
            Div(
                Label(
                    Span(
                        "Select Provider",
                        cls=combine_classes(
                            font_weight.medium,
                            text_dui.base_content,
                            font_size.sm  # Slightly smaller for hierarchy
                        )
                    ),
                    Select(
                        *[
                            Option(
                                format_provider_name(p, provider_config),
                                value=p,
                                selected=(p == default)
                            )
                            for p in providers
                        ],
                        name="provider",
                        cls=combine_classes(
                            select,
                            select_sizes.md,  # Consistent sizing
                            w.full,
                            bg_dui.base_100,
                            border_dui.base_300,  # Subtle border
                            transition.all,
                            duration._200,
                            ring(2).focus,
                            ring_dui.primary.opacity(50).focus
                        ),
                        required=True,
                        id="provider-select",
                        onchange="updatePlaceholder(this)"
                    ),
                    cls=combine_classes(flex_display, flex_direction.col, gap(2))
                )
            ),
            
            # API key input with enhanced design
            Div(
                Label(
                    Span(
                        "API Key",
                        cls=combine_classes(
                            font_weight.medium,
                            text_dui.base_content,
                            font_size.sm
                        )
                    ),
                    Input(
                        type="password",
                        name="api_key",
                        placeholder=get_provider_info(default, provider_config)['placeholder'] if default else "Enter your API key",
                        cls=combine_classes(
                            text_input,
                            text_input_sizes.md,
                            w.full,
                            bg_dui.base_100,
                            transition.all,
                            duration._200,
                            ring(2).focus,
                            ring_dui.primary.opacity(50).focus
                        ),
                        required=True,
                        id="api-key-input",
                        autocomplete="off"
                    ),
                    cls=combine_classes(flex_display, flex_direction.col, gap(2))
                ),
                P(
                    "Your API key will be encrypted and stored securely",
                    cls=combine_classes(
                        font_size.sm,
                        text_dui.base_content.opacity(60),
                        m.t(1)
                    )
                )
            ),
            cls=combine_classes(
                flex_display,
                flex_direction.col,
                gap(6),
                p(6),  # Internal padding
                bg_dui.base_200,  # Subtle background
                rounded.lg,  # Rounded corners
                border,
                border_dui.base_300  # Subtle border
            )
        ),
        
        # Submit button with better prominence
        Button(
            "Save Key",
            type="submit",
            cls=combine_classes(
                btn,
                btn_colors.primary,
                btn_sizes.md,
                w.full,
                shadow.md,
                transition.all,
                duration._200,
                shadow.lg.hover,
                m.t(2)  # Spacing from form
            )
        ),
        
        method=method,
        action=action,
        cls=combine_classes(
            flex_display,
            flex_direction.col,
            gap(4),
            max_w.lg,  # Wider for provider selection
            w.full
        )
    )

In [ ]:
#| eval: false
#| output: false
test_func = MultiProviderKeyForm
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Test with custom provider names
    providers = ['my_llm_api', 'custom_service', 'another_provider']
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), 
                           test_func(providers))
server = start_test_server(app)
display(HTMX())
server.stop()

## Key Management Card

In [ ]:
#| export
def KeyManagementCard(
    provider: str,  # Provider identifier
    has_key: bool,  # Whether a key is stored
    masked_key: Optional[str] = None,  # Masked version of the key for display
    created_at: Optional[str] = None,  # When the key was stored
    expires_at: Optional[str] = None,  # When the key expires
    delete_action: Optional[str] = None,  # URL for delete action
    update_action: Optional[str] = None,  # URL for update action
    provider_config: Optional[Dict[str, Any]] = None  # Optional provider configuration
) -> Div:  # Card component for key management
    "Create a card component for managing a stored API key with enhanced design."
    provider_info = get_provider_info(provider, provider_config)
    
    card_content = []
    
    # Enhanced title with better typography
    card_content.append(
        Div(
            provider_info['name'],
            cls=combine_classes(
                card_title,
                text_dui.base_content,
                font_size.lg,  # Slightly smaller for card context
                font_weight.semibold
            )
        )
    )
    
    if has_key:
        # Key info with better visual hierarchy
        if masked_key:
            card_content.append(
                P(
                    Span("Current Key: ", cls=combine_classes(
                        text_dui.base_content.opacity(70),
                        font_size.sm
                    )),
                    Span(
                        masked_key,
                        cls=combine_classes(
                            font_family.mono,
                            text_dui.primary,  # Use primary color for key
                            font_size.sm,
                            bg_dui.base_200,  # Subtle background
                            p.x(2),
                            p.y(1),
                            rounded.md
                        )
                    )
                )
            )
        
        # Metadata with consistent styling
        if created_at:
            card_content.append(
                P(
                    f"Added: {created_at}",
                    cls=combine_classes(
                        font_size.sm,
                        text_dui.base_content.opacity(60)
                    )
                )
            )
        
        if expires_at:
            card_content.append(
                P(
                    f"Expires: {expires_at}",
                    cls=combine_classes(
                        font_size.sm,
                        text_dui.warning,  # Warning color for expiration
                        font_weight.medium
                    )
                )
            )
        
        # Enhanced action buttons
        actions = []
        
        if update_action:
            actions.append(
                Form(
                    Div(
                        Input(
                            type="password",
                            name="api_key",
                            placeholder=provider_info['placeholder'],
                            cls=combine_classes(
                                text_input,
                                text_input_sizes.sm,
                                w.full,  # Full width within container
                                transition.all,
                                duration._200
                            ),
                            required=True
                        ),
                        Button(
                            "Update",
                            type="submit",
                            cls=combine_classes(
                                btn,
                                btn_colors.primary,
                                btn_sizes.sm,
                                transition.all,
                                duration._200
                            )
                        ),
                        cls=combine_classes(
                            flex_display, 
                            gap(2), 
                            items.end,
                            w.full,  # Container takes full width
                            max_w(300)  # But limit maximum width
                        )
                    ),
                    method="post",
                    action=update_action,
                    cls=str(w.full)  # Form takes full width of its container
                )
            )
        
        if delete_action:
            actions.append(
                Form(
                    Button(
                        "Delete",
                        type="submit",
                        cls=combine_classes(
                            btn,
                            btn_colors.error,
                            btn_sizes.sm,
                            transition.all,
                            duration._200,
                            bg_dui.error.hover,  # Fill on hover
                            text_dui.error_content.hover
                        ),
                        onclick="return confirm('Are you sure you want to delete this key?');"
                    ),
                    method="post",
                    action=delete_action
                )
            )
        
        if actions:
            card_content.append(
                Div(
                    *actions,
                    cls=combine_classes(
                        card_actions,
                        justify.between,  # Space between actions
                        flex_direction.col.sm,  # Stack on mobile
                        flex_direction.row.md,  # Row on desktop
                        gap(2),
                        w.full  # Ensure full width
                    )
                )
            )
    else:
        # No key state with better messaging
        card_content.append(
            Div(
                P(
                    "No API key configured",
                    cls=combine_classes(
                        font_size.sm,
                        text_dui.base_content.opacity(60),
                        m.b(4)
                    )
                ),
                # Inline add key form
                KeyInputForm(
                    provider=provider,
                    action=update_action,
                    show_help=False,
                    provider_config=provider_config
                ) if update_action else None,
                cls=combine_classes(space.y(4))
            )
        )
    
    return Div(
        Div(
            *card_content,
            cls=str(card_body)
        ),
        cls=combine_classes(
            card,
            bg_dui.base_100,  # Ensure proper background
            shadow.md,  # Add depth
            transition.all,
            duration._200,
            shadow.lg.hover,  # Elevate on hover
            border,
            border_dui.base_300,  # Subtle border
            w.full  # Ensure card takes full width of its grid cell
        )
    )

In [ ]:
#| eval: false
#| output: false
test_func = KeyManagementCard
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Test with custom provider and optional config
    custom_config = {
        'api_service': {
            'name': 'My API Service',
            'placeholder': 'Enter your API key here...'
        }
    }
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), 
                           test_func(
                               provider="api_service",
                               has_key=True,
                               masked_key="key-...xyz",
                               created_at="2 hours ago",
                               provider_config=custom_config
                           ))
server = start_test_server(app)
display(HTMX())
server.stop()

## Key Manager Dashboard

In [ ]:
#| export
def KeyManagerDashboard(
    request,  # FastHTML request object
    providers: List[str],  # List of provider identifiers to manage
    byok_manager = None,
    user_id: Optional[str] = None,  # Optional user ID for database storage
    base_url: str = "/api/keys",  # Base URL for API endpoints
    provider_config: Optional[Dict[str, Any]] = None  # Optional provider configuration
) -> Div:  # Dashboard component with all provider cards
    "Create a complete dashboard for managing multiple API keys with improved layout."
    # Import responsive grid helper
    from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import grid_display, grid_cols
    from cjm_fasthtml_tailwind.utilities.sizing import min_h, h, container
    
    # Get BYOK manager from request scope if not provided
    if byok_manager is None and 'byok' in request.scope:
        byok_manager = request.scope['byok']
    
    cards = []
    
    for provider in providers:
        has_key = False
        masked_key = None
        
        if byok_manager:
            # Check if key exists
            has_key = byok_manager.has_key(request, provider, user_id)
            if has_key:
                # Get masked version of key
                actual_key = byok_manager.get_key(request, provider, user_id)
                if actual_key:
                    masked_key = mask_key(actual_key)
        
        cards.append(
            KeyManagementCard(
                provider=provider,
                has_key=has_key,
                masked_key=masked_key,
                delete_action=f"{base_url}/{provider}/delete" if has_key else None,
                update_action=f"{base_url}/{provider}",
                provider_config=provider_config
            )
        )
    
    return Div(
        # Container for proper max-width and centering
        Div(
            # Header section with better spacing
            Div(
                Div(
                    "API Key Management",
                    cls=combine_classes(
                        font_size._2xl,
                        font_weight.bold,
                        text_dui.base_content
                    )
                ),
                P(
                    "Manage your API keys for different providers",
                    cls=combine_classes(
                        text_dui.base_content.opacity(70),
                        m.t(2)
                    )
                ),
                cls=combine_classes(m.b(8))
            ),
            
            # Cards grid - matching the test layout exactly
            Div(
                *cards,
                cls=combine_classes(
                    grid_display,
                    grid_cols(1),  # 1 column on mobile
                    grid_cols(2).md,  # 2 columns on tablet
                    grid_cols(3).lg,  # 3 columns on desktop
                    gap(4),  # Consistent gap matching test
                    w.full
                )
            ),
            cls=combine_classes(
                container,  # Container for proper max-width
                m.x.auto,  # Center the container
                p(4),  # Padding for mobile
                p(6).md  # More padding on larger screens
            )
        ),
        cls=combine_classes(
            w.full,
            min_h.screen,  # Full height
            bg_dui.base_200.opacity(10)  # Very subtle background
        )
    )

In [ ]:
#| eval: false
#| output: false
# Test using the actual KeyManagerDashboard function
test_func = KeyManagerDashboard
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Create a mock request object
    class MockRequest:
        def __init__(self):
            self.scope = {}
    
    request = MockRequest()
    
    # Test with custom providers and optional config
    providers = ["llm_service", "embedding_api", "vector_db", "search_api", "analytics", "monitoring"]
    
    # Optional: provide custom config for some providers
    provider_config = {
        'llm_service': {
            'name': 'LLM Service',
            'placeholder': 'Enter your LLM API key'
        },
        'embedding_api': {
            'name': 'Embedding API',
            'placeholder': 'Your embedding API key'
        }
        # Other providers will use auto-generated defaults
    }
    
    # Create the dashboard using the actual function
    dashboard = KeyManagerDashboard(
        request=request,
        providers=providers,
        byok_manager=None,  # No manager, so all will show as no key
        base_url="/api/keys",
        provider_config=provider_config
    )
    
    return create_test_page("Key Manager Dashboard (Using Function)", dashboard)

server = start_test_server(app)
display(HTMX())
server.stop()

## Inline Key Input

In [ ]:
#| export
def InlineKeyInput(
    provider: str,  # Provider identifier
    input_id: Optional[str] = None,  # HTML ID for the input element
    on_save: Optional[str] = None,  # JavaScript to execute on save (or hx-post URL for HTMX)
    compact: bool = True,  # Whether to use compact styling
    provider_config: Optional[Dict[str, Any]] = None  # Optional provider configuration
) -> Div:  # Inline input component
    "Create a compact inline key input component with polished design."
    provider_info = get_provider_info(provider, provider_config)
    input_id = input_id or f"key-input-{provider}"
    
    # Import ring_dui for semantic ring colors
    from cjm_fasthtml_daisyui.utilities.semantic_colors import ring_dui
    
    input_elem = Input(
        type="password",
        placeholder=provider_info['placeholder'],
        cls=combine_classes(
            text_input,
            text_input_sizes.sm if compact else text_input_sizes.md,
            bg_dui.base_100,
            border_dui.base_300,
            transition.all,
            duration._200,
            ring(2).focus,
            ring_dui.primary.opacity(50).focus,  # Use semantic ring color
            grow()  # Allow input to grow
        ),
        id=input_id,
        name="api_key"
    )
    
    save_btn = Button(
        "Save",
        cls=combine_classes(
            btn,
            btn_colors.primary,
            btn_sizes.sm if compact else btn_sizes.md,
            shadow.sm,
            transition.all,
            duration._200,
            shadow.md.hover
        )
    )
    
    # Add HTMX attributes if on_save looks like a URL
    if on_save and on_save.startswith('/'):
        save_btn.attrs['hx-post'] = on_save
        save_btn.attrs['hx-trigger'] = 'click'
        save_btn.attrs['hx-include'] = f'#{input_id}'
    elif on_save:
        save_btn.attrs['onclick'] = on_save
    
    return Div(
        input_elem,
        save_btn,
        cls=combine_classes(
            flex_display,
            gap(2),
            items.stretch,  # Align items properly
            w.full if not compact else None
        )
    )

In [ ]:
#| eval: false
#| output: false
test_func = InlineKeyInput
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Test with custom provider
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), 
                           test_func(
                               provider="custom_backend",
                               on_save="/api/keys/custom_backend/quick-save"
                           ))
server = start_test_server(app)
display(HTMX())
server.stop()

## Tests

In [ ]:
# Test basic key input form with custom provider
form = KeyInputForm(provider="my_service")
assert form.tag == "form"
assert form.attrs['method'] == "post"
assert form.attrs['action'] == "/api/keys/my_service"
print("✓ Basic key input form")

# Test with provider config
config = {
    'openai': {
        'name': 'OpenAI GPT',
        'placeholder': 'sk-...'
    }
}
form_with_config = KeyInputForm(provider="openai", provider_config=config)
assert form_with_config.tag == "form"
print("✓ Key input form with config")

# Test multi-provider form
multi_form = MultiProviderKeyForm(
    providers=["service_a", "service_b", "service_c"],
    default_provider="service_b"
)
assert multi_form.tag == "form"
print("✓ Multi-provider form")

# Test key management card with custom provider
card_with_key = KeyManagementCard(
    provider="custom_api",
    has_key=True,
    masked_key="key-...abc",
    created_at="2 hours ago"
)
assert card_with_key.tag == "div"
print("✓ Key management card with key")

card_without_key = KeyManagementCard(
    provider="another_service",
    has_key=False,
    update_action="/api/keys/another_service"
)
assert card_without_key.tag == "div"
print("✓ Key management card without key")

# Test inline input with custom provider
inline = InlineKeyInput(
    provider="my_llm_service",
    on_save="/api/keys/my_llm_service/quick-save"
)
assert inline.tag == "div"
print("✓ Inline key input")

print("\nAll form component tests passed!")

✓ Basic key input form
✓ Key input form with config
✓ Multi-provider form
✓ Key management card with key
✓ Key management card without key
✓ Inline key input

All form component tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()